

 <h1>
Welcome to the Math Question Answer Verification Competition! 🚀

The goal is to fine-tune a Llama-3-8B model to predict if a given solution to a math problem is correct or not. Your model should output True if the solution is correct, and False otherwise.

This notebook is a starter guide designed to get you up and running quickly. We'll walk through a simplified training process using a small subset of the data (5,000 examples) and lightweight parameters. The main goal here is to understand the complete workflow, from loading data to generating a submission file, not to achieve a top score.

Good luck, and have fun! 🎉

## **Step 1: Install Necessary Libraries**

First, we need to install the required Python libraries. We'll be using the unsloth library, which provides highly efficient, memory-saving training methods for large language models, making it possible to fine-tune powerful models on a single free-tier GPU. We'll also install xformers for further optimization.


In [ ]:
# @title
# %%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
# !pip install --no-deps "xformers<0.0.26" "trl<0.9.0" "peft<0.12.0" "accelerate<0.32.0" "bitsandbytes<0.44.0" "transformers<4.43.0"

  Resolved https://github.com/unslothai/unsloth.git to commit d707bd43b4e883b521761d525be2fae428fe5980
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 173.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.6/273.6 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 166.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
%pip uninstall -y unsloth unsloth_zoo trl
%pip install --upgrade --force-reinstall --no-cache-dir "trl>=0.14,<0.17" unsloth unsloth_zoo


Found existing installation: unsloth 2025.10.12
Uninstalling unsloth-2025.10.12:
  Successfully uninstalled unsloth-2025.10.12
Found existing installation: unsloth_zoo 2025.10.13
Uninstalling unsloth_zoo-2025.10.13:
  Successfully uninstalled unsloth_zoo-2025.10.13
Found existing installation: trl 0.23.0
Uninstalling trl-0.23.0:
  Successfully uninstalled trl-0.23.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 241.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 291.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 275.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of unsloth to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 286.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 292.4 MB/s eta 0:00:00
 

In [ ]:
import IPython, sys
print("Installed. Python:", sys.version)
print("Please now go to: Runtime -> Restart runtime, then re-run the next cell.")

Installed. Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Please now go to: Runtime -> Restart runtime, then re-run the next cell.


In [ ]:
import sys
print("Python:", sys.version)

import unsloth, transformers, trl
print("unsloth =", getattr(unsloth, "__version__", "?"))
print("transformers =", transformers.__version__)
print("trl =", trl.__version__)

Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


ImportError: Unsloth: Please install unsloth_zoo via `pip install unsloth_zoo`

## **Step 2: Load the Model and Tokenizer**

Next, we'll load the Llama-3-8B model, which is the only model permitted for this competition. We'll use Unsloth's FastLanguageModel to handle this efficiently.

A key technique we'll use is 4-bit quantization (load_in_4bit = True). Think of this as compressing the model's knowledge into a much smaller file size. This significantly reduces the amount of GPU memory required, allowing us to fine-tune this large model even on a free platform like Google Colab.



In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 1024  # Choose any sequence length
dtype = None  # This will auto-detect the best data type for your GPU
load_in_4bit = True  # Use 4-bit quantization to save memory

# Load the model and tokenizer from Hugging Face
# Note: We use the base model, not a 4-bit pre-quantized one,
# to ensure we start from the official weights.
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B", # Competition-approved model
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Tokenizer safety/commons
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

## **Step 3: Prepare the Dataset**

This is a crucial step where we format our data into a structure the model can learn from. The process involves three parts:

1.  **Loading**: We'll load the official competition dataset from Hugging Face.
2.  **Splitting**: The full dataset is massive. For this starter notebook, we'll create a much smaller, more manageable version to speed things up: **5,000 samples for training** and **500 for validation**.
3.  **Prompting**: We will format each data sample into a clear instructional prompt. This helps the model understand its role as a mathematician verifying a solution.



In [ ]:
from datasets import load_dataset

# Load the full training dataset
full_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="train")

# Shuffle the dataset for randomness and create our smaller splits
shuffled_dataset = full_dataset.shuffle(seed=42)
train_dataset = shuffled_dataset.select(range(5000))      # Use the first 5,000 for training
validation_dataset = shuffled_dataset.select(range(5000, 5500)) # Use the next 500 for validation

## **Step 4: Configure LoRA and Set Up the Trainer**

### **LoRA Configuration**

Instead of training the entire model (which has billions of parameters), we'll use a technique called **Lo**w-**R**ank **A**daptation (LoRA). 🎛️

Think of it like this: rather than rewriting an entire textbook, we're just adding small, efficient "sticky notes" (the LoRA adapters) to update the model's knowledge. This is much faster and requires significantly less memory. We'll use a small **rank** (`r = 8`) to keep the training process light and quick for this starter notebook.


In [ ]:
import torch
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    load_in_4bit=True,
)

# The instructional prompt template for training
training_prompt = """You are a great mathematician and you are tasked with finding if a solution to a given maths question is correct or not. Your response should be 'True' if the solution is correct, otherwise 'False'. Below is the Question and Solution.
Question:
{}
Solution:
{}
Output:
{}"""

# We must add an End Of Sequence (EOS) token to tell the model when a completion is finished.
EOS_TOKEN = tokenizer.eos_token

# This function formats our data samples into the prompt template.
def formatting_prompts_func(examples):
    questions = examples["question"]
    solutions = examples["solution"]
    outputs = examples["is_correct"]
    texts = []
    for question, solution, output in zip(questions, solutions, outputs):
        # Format the prompt and add the EOS token
        text = training_prompt.format(question, str(solution), str(output)) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts }

# Apply the formatting function to our training dataset
formatted_train_dataset = train_dataset.map(formatting_prompts_func, batched=True)

In [ ]:
from peft import PeftModel
import torch

# 1) If model already has LoRA, merge & remove them
if isinstance(model, PeftModel) or getattr(model, "peft_type", None) is not None:
    # This folds current LoRA into the base weights and detaches PEFT wrappers
    model = model.merge_and_unload()
    torch.cuda.empty_cache() if torch.cuda.is_available() else None

# 2) Now you can safely attach your new LoRA config
model = FastLanguageModel.get_peft_model(
    model,
    r=32,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    lora_alpha=64,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42,
)

# 3) (Your TRUE/FALSE helper stays the same)
def _stable_id(tok, text: str):
    ids = tok.encode(text, add_special_tokens=False)
    if not ids:
        ids = tok.encode(" " + text, add_special_tokens=False)
    return ids[-1] if ids else tok.eos_token_id

TRUE_ID  = _stable_id(tokenizer, "True")
FALSE_ID = _stable_id(tokenizer, "False")
PAD_ID   = tokenizer.pad_token_id
EOS_ID   = tokenizer.eos_token_id

In [ ]:
def get_stable_id(tokenizer, text: str):
    ids = tokenizer.encode(text, add_special_tokens=False)
    if not ids:
        ids = tokenizer.encode(" " + text, add_special_tokens=False)
    return ids[-1] if ids else tokenizer.eos_token_id

    TRUE_ID  = get_stable_id(tokenizer, "True")
FALSE_ID = get_stable_id(tokenizer, "False")
ALLOWED_IDS = [TRUE_ID, FALSE_ID]

class KeepOnlyIdsProcessor(torch.nn.Module):
    def __init__(self, allowed_ids):
        super().__init__()
        self.allowed = set(int(i) for i in allowed_ids)
    def forward(self, input_ids, scores):
        mask = torch.full_like(scores, float("-inf"))
        mask[:, list(self.allowed)] = 0.0
        return scores + mask

if hasattr(tokenizer, "padding_side"):
    tokenizer.padding_side = "left"

In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=2, shuffle=True, random_state=42)
fold_losses = []

for fold, (train_idx, val_idx) in enumerate(kf.split(train_dataset)):
    print(f"\n=== Fold {fold+1}/2 ===")

    # Split the dataset
    train_fold = shuffled_dataset.select(train_idx.tolist())
    val_fold   = shuffled_dataset.select(val_idx.tolist())

    # Apply your formatting function
    formatted_train = train_fold.map(formatting_prompts_func, batched=True, remove_columns=train_fold.column_names)
    formatted_val   = val_fold.map(formatting_prompts_func, batched=True, remove_columns=val_fold.column_names)

    # Update trainer’s datasets
    trainer.train_dataset = formatted_train
    trainer.eval_dataset  = formatted_val

    trainer.train()
    metrics = trainer.evaluate()
    fold_losses.append(metrics["eval_loss"])

print(f"Mean eval loss: {sum(fold_losses)/len(fold_losses):.4f}")



### **SFTTrainer Setup**

Now we'll set up the `SFTTrainer` (Supervised Fine-tuning Trainer). This is the main tool from the `trl` library that will handle the entire training loop for us. We'll give it our model, tokenizer, dataset, and a set of training instructions, such as the batch size and number of epochs.

We will train for just **one epoch** (a single pass over our 5,000-sample dataset) to keep this demonstration fast.

In [ ]:
from transformers import EarlyStoppingCallback

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_train_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,
        learning_rate=1.5e-4,
        fp16=True,
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        warmup_ratio=0.1,                   # gradual LR increase
        lr_scheduler_type="cosine_with_restarts",  # cyclic decay
        output_dir="outputs",
        evaluation_strategy="steps",
        eval_steps=20,
        save_total_limit=2,
        load_best_model_at_end=True,
        report_to="none",
    ),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)


## **Step 5: Start Training\!**

Now, we'll call the `train()` function on our `trainer` object. This will kick off the fine-tuning process. Based on our settings, this will run for one full epoch over our 5,000 examples.

Grab a coffee, as this will take a few minutes\! ☕


In [ ]:
from torch.nn.utils import clip_grad_norm_

def clip_gradients(model, max_norm=1.0):
    clip_grad_norm_(model.parameters(), max_norm)


In [ ]:
trainer.train()
# train_result = trainer.train()
# trainer.save_state()
# trainer.save_model("outputs/lora_adapter")   # saves LoRA weights & trainer state
# tokenizer.save_pretrained("outputs/lora_adapter")

# print("Training done. Steps:", train_result.global_step)

In [ ]:
clip_gradients(model)



## **Step 6: Inference and Evaluation**

Now that our model is trained, we need to test it on our validation set. We'll use a slightly different prompt for inference—one where we leave the `Output:` section blank for the model to complete.

Let's test it on a single example from our validation set to see what it predicts.

In [ ]:
import torch

model.eval()
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token
PAD_ID = tokenizer.pad_token_id
EOS_ID = tokenizer.eos_token_id
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# --- helper: get a stable ID for the word True/False (handles tokenizers that split) ---
def last_id_for(text: str) -> int:
    ids = tokenizer.encode(text, add_special_tokens=False)
    # fallback: try a leading space if it splits differently
    if len(ids) == 0:
        ids = tokenizer.encode(" " + text, add_special_tokens=False)
    return ids[-1] if len(ids) > 0 else tokenizer.eos_token_id

TRUE_ID  = last_id_for("True")
FALSE_ID = last_id_for("False")

# ✅ Define your inference prompt FIRST
inference_prompt = """You are a great mathematician and you are tasked with finding
if a solution to a given maths question is correct or not.
Your response should be 'True' if the solution is correct,
otherwise 'False'.

Question:
{}
Solution:
{}
Answer:"""

# ---- Example data (for testing) ----
# If these are not already defined earlier, add them temporarily for test
example = validation_dataset[10]  # or any example
question = example["question"]
solution = example["solution"]

# ---- Tokenize ----
inputs = tokenizer(
    [inference_prompt.format(question, str(solution))],
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=max_seq_length if "max_seq_length" in globals() else 2048,
).to(device)

# ---- Logit comparison for True/False ----
with torch.inference_mode():
    logits = model(**inputs).logits[:, -1, :]          # [batch, vocab]
    probs  = torch.softmax(logits, dim=-1)

    p_true  = probs[0, TRUE_ID].item()
    p_false = probs[0, FALSE_ID].item()

    pred_bool = p_true > p_false
    pred_text = "True" if pred_bool else "False"

# ---- Print results ----
print("#### QUESTION ####")
print(question)
print("\n#### SOLUTION ####")
print(solution)
print("\n#### MODEL'S PREDICTION ####")
print(pred_text, f"(p_true={p_true:.4f}, p_false={p_false:.4f})")

print("\n#### CORRECT ANSWER ####")
print(example["is_correct"])


## **Step 7: Generate Submission File**

This is the final step\! We will now run our fine-tuned model on the official `test` dataset.

We will loop through each example in the test set, generate a prediction, and format the results into a CSV file with two columns: `ID` and `is_correct`, as required by the competition.


In [ ]:

# === Cell A: Validation-based threshold calibration ===
import torch, numpy as np

model.eval()
device = next(model.parameters()).device
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token
PAD_ID = tokenizer.pad_token_id
EOS_ID = tokenizer.eos_token_id

def last_id_for(text: str) -> int:
    ids = tokenizer.encode(text, add_special_tokens=False)
    if not ids:
        ids = tokenizer.encode(" " + text, add_special_tokens=False)
    return ids[-1] if ids else tokenizer.eos_token_id

TRUE_ID  = last_id_for("True")
FALSE_ID = last_id_for("False")

PROMPT_A = "You are a careful math verifier. Answer strictly with 'True' if the solution is correct, otherwise 'False'.\nQuestion:\n{}\nSolution:\n{}\nAnswer:"
PROMPT_B = "You are a careful math verifier. Reply only with 'False' if the solution is wrong, otherwise 'True'.\nQuestion:\n{}\nSolution:\n{}\nAnswer:"

@torch.no_grad()
def prob_true_from_logits(prompt: str, samples:int=3, temp:float=0.3):
    probs_true = []
    for _ in range(samples):
        inputs = tokenizer([prompt], return_tensors="pt", padding=True, truncation=True, max_length=2048).to(device)
        logits = model(**inputs).logits[:, -1, :]
        if temp and temp > 0:
            logits = logits / temp
        scores = torch.softmax(logits, dim=-1)
        p_t = scores[0, TRUE_ID].item()
        p_f = scores[0, FALSE_ID].item()
        denom = (p_t + p_f) if (p_t + p_f) > 0 else 1.0
        probs_true.append(p_t / denom)
    return float(sum(probs_true) / len(probs_true))

val_N = len(validation_dataset)
val_probs, val_labels = [], []

for i in range(val_N):
    ex = validation_dataset[i]
    q, s, y = ex["question"], ex["solution"], bool(ex["is_correct"])
    pA = prob_true_from_logits(PROMPT_A.format(q, str(s)), samples=3, temp=0.3)
    pB = prob_true_from_logits(PROMPT_B.format(q, str(s)), samples=3, temp=0.3)
    p = (pA + pB) / 2.0
    val_probs.append(p)
    val_labels.append(y)

import numpy as np
best_thr, best_acc = 0.5, 0.0
probs_arr = np.array(val_probs, dtype=float)
labels_arr = np.array(val_labels, dtype=bool)
for thr in np.linspace(0.2, 0.8, 121):
    preds = probs_arr >= thr
    acc = (preds == labels_arr).mean()
    if acc > best_acc:
        best_acc, best_thr = acc, float(thr)

print(f"✅ Calibrated threshold: {best_thr:.3f} | Val accuracy: {best_acc:.4f}")
CALIB_THRESHOLD = best_thr
TRUE_ID_GLOBAL = TRUE_ID
FALSE_ID_GLOBAL = FALSE_ID
PROMPT_A_GLOBAL = PROMPT_A
PROMPT_B_GLOBAL = PROMPT_B


In [ ]:
# === Cell B: Resilient 10K inference with logits + calibrated threshold ===
import os, json, gc
import torch
import pandas as pd
from datasets import load_dataset

CHECKPOINT_CSV = "submission_partial.csv"
STATE_JSON     = "inference_state.json"
FINAL_CSV      = "submission.csv"
CHUNK_SIZE     = 1000
SAVE_EVERY     = 100
PRINT_EVERY    = 100
EMPTY_CACHE_EVERY = 200
MAX_LEN        = 2048

# These must be defined earlier in the notebook (as you had them):
# TRUE_ID_GLOBAL, FALSE_ID_GLOBAL, PROMPT_A_GLOBAL, PROMPT_B_GLOBAL, CALIB_THRESHOLD
TRUE_ID  = TRUE_ID_GLOBAL
FALSE_ID = FALSE_ID_GLOBAL
PROMPT_A = PROMPT_A_GLOBAL
PROMPT_B = PROMPT_B_GLOBAL

torch.set_grad_enabled(False)
model.eval()
device = next(model.parameters()).device
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token
PAD_ID = tokenizer.pad_token_id
EOS_ID = tokenizer.eos_token_id

dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="test")
N = len(dataset)

@torch.no_grad()
def prob_true_from_logits(prompt: str, samples: int = 3, temp: float = 0.3) -> float:
    ps = []
    for _ in range(samples):
        inputs = tokenizer([prompt], return_tensors="pt",
                           padding=True, truncation=True, max_length=MAX_LEN).to(device)
        logits = model(**inputs).logits[:, -1, :]  # next-token logits
        if temp and temp > 0:
            logits = logits / temp
        scores = torch.softmax(logits, dim=-1)
        p_t = scores[0, TRUE_ID].item()
        p_f = scores[0, FALSE_ID].item()
        denom = (p_t + p_f) if (p_t + p_f) > 0 else 1.0
        ps.append(p_t / denom)
    return float(sum(ps) / len(ps))

def predict_one_prob(question: str, solution: str, samples: int = 3, temp: float = 0.3) -> float:
    try:
        pA = prob_true_from_logits(PROMPT_A.format(question, str(solution)), samples=samples, temp=temp)
    except Exception:
        pA = 0.5
    try:
        pB = prob_true_from_logits(PROMPT_B.format(question, str(solution)), samples=samples, temp=temp)
    except Exception:
        pB = 0.5
    return (pA + pB) / 2.0

def save_checkpoint(df: pd.DataFrame, upto_idx: int) -> None:
    df.to_csv(CHECKPOINT_CSV, index=False)
    with open(STATE_JSON, "w") as f:
        json.dump({"next_start": int(upto_idx)}, f)

def load_checkpoint():
    if os.path.exists(CHECKPOINT_CSV) and os.path.exists(STATE_JSON):
        try:
            df = pd.read_csv(CHECKPOINT_CSV)
            with open(STATE_JSON, "r") as f:
                st = json.load(f)
            next_start = int(st.get("next_start", len(df)))
            if len(df) == next_start:
                return df, next_start
        except Exception:
            pass
    return None, 0

partial_df, start_idx = load_checkpoint()
if partial_df is None:
    partial_df = pd.DataFrame({"ID": [], "is_correct": []})
predictions = partial_df["is_correct"].tolist() if len(partial_df) else []

print(f"✅ Resume state: {start_idx} / {N}")

errors = 0
for chunk_start in range(start_idx, N, CHUNK_SIZE):
    chunk_end = min(chunk_start + CHUNK_SIZE, N)
    print(f"\n--- Processing {chunk_start} .. {chunk_end-1} ---")

    for i in range(chunk_start, chunk_end):
        try:
            ex = dataset[i]
            q, s = ex["question"], ex["solution"]
            p_true = predict_one_prob(q, s, samples=3, temp=0.3)
            pred_bool = p_true >= CALIB_THRESHOLD
            predictions.append(bool(pred_bool))
        except Exception:
            errors += 1
            predictions.append(False)

        processed = i + 1
        if processed % PRINT_EVERY == 0:
            print(f"Processed {processed}/{N} (errors so far: {errors})")
        if processed % SAVE_EVERY == 0:
            tmp = pd.DataFrame({"ID": range(len(predictions)), "is_correct": predictions})
            save_checkpoint(tmp, processed)
        if processed % EMPTY_CACHE_EVERY == 0:
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            gc.collect()

    tmp = pd.DataFrame({"ID": range(len(predictions)), "is_correct": predictions})
    save_checkpoint(tmp, chunk_end)
    print(f"✓ Chunk {chunk_start}-{chunk_end-1} saved. Total rows now: {len(predictions)}")

final_df = pd.DataFrame({"ID": range(len(predictions)), "is_correct": predictions})
final_df.to_csv(FINAL_CSV, index=False)

# Optional: clean up checkpoint files after success
try:
    os.remove(CHECKPOINT_CSV)
    os.remove(STATE_JSON)
except Exception:
    pass

print("\n🎉 Done.")
print(f"Total rows: {N} | Predictions: {len(predictions)} | Per-sample errors recovered: {errors}")
print(f"Threshold used: {CALIB_THRESHOLD:.3f}")
print(f"Final file: {FINAL_CSV}")
